In [14]:
import telegram
import requests
import telegram_send

import schedule
import time
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from bs4 import BeautifulSoup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
#from bob_telegram_tools.bot import TelegramBot


In [15]:
token = '1796099831:AAGteQJXlcNd2dLSSq0kbbqa0QoV0B_xtTU'
    
bot = telegram.Bot(token) #Replace TOKEN with your token string
updater = Updater(token=token, use_context=True) #Replace TOKEN with your token string

dispatcher = updater.dispatcher

In [16]:
def get_coin_bid(coin):
    response = requests.get('https://economia.awesomeapi.com.br/json/last/'+ coin)
    result = response.json()
    c = coin.replace('-','')
    return result[c]['bid']

In [17]:
def get_bdrs_price(paper):
    response = 'https://statusinvest.com.br/bdrs/' + paper
    result = requests.get(response)
    soup = BeautifulSoup(result.content, 'html.parser')
    stock_price = str(soup.find(class_='value')).split('>')[1].split('<')[0]
    var = str(soup.find(title='Variação do valor do ativo com base no dia anterior'))
    
    var = var.split('\n')[4].split('>')[1].split('<')[0]
        
    if not var.startswith('-'):
        var = '+' + var
            
    return stock_price, var

In [18]:
def get_fiis_price(paper):
    response = 'https://statusinvest.com.br/fundos-imobiliarios/' + paper
    result = requests.get(response)
    soup = BeautifulSoup(result.content, 'html.parser')
    stock_price = str(soup.find(class_='value'))
    var = str(soup.find(title='Variação do valor do ativo com base no dia anterior'))
    
    if stock_price == 'None':
        return get_bdrs_price(paper)
    else: 
        stock_price = stock_price.split('>')[1].split('<')[0]
        var = var.split('\n')[4].split('>')[1].split('<')[0]
        
        if not var.startswith('-'):
            var = '+' + var
            
        return stock_price, var
    

In [19]:
def get_stock_price(paper):
    response = 'https://statusinvest.com.br/acoes/' + paper
    result = requests.get(response)
    soup = BeautifulSoup(result.content, 'html.parser')
    stock_price = str(soup.find(class_='value'))
    var = str(soup.find(title='Variação do valor do ativo com base no dia anterior'))
    
    if stock_price == 'None':
        return get_fiis_price(paper)
    else: 
        
        stock_price = stock_price.split('>')[1].split('<')[0]
        var = var.split('\n')[4].split('>')[1].split('<')[0]
        
        if not var.startswith('-'):
            var = '+' + var
            
        return stock_price, var
    
    
    
    

In [20]:
def stock(update, context):
    paper = " ".join(context.args)
    value = get_stock_price(paper.lower())
    update.message.reply_text(paper.upper() + ": R$" + value[0] + '\n \n Variação do dia: ' + value[1])

In [21]:
def euro(update, context):
    eur = "Cotação do Euro Hoje: R$ " + str(get_coin_bid('EUR-BRL')).replace('.',',')
    context.bot.send_message(chat_id=update.effective_chat.id, text=eur)

In [22]:
def vilzyn(update, context):
    dol = "Cotação do Dolar Hoje: R$ " + str(get_coin_bid('USD-BRL')).replace('.',',')
    context.bot.send_message(chat_id=update.effective_chat.id, text=dol)

In [23]:
def dolar(update, context):
    dol = "Cotação do Dolar Hoje: R$ " + str(get_coin_bid('USD-BRL')).replace('.',',')
    context.bot.send_message(chat_id=update.effective_chat.id, text=dol)

In [24]:
def hello(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text='Eai Kamako')

In [25]:
def chart(update, context):
    paper = " ".join(context.args)
    get_stock_chart(paper)
    context.bot.sendPhoto(chat_id=update.effective_chat.id, photo=open('paper' + '.png', 'rb'))
    os.remove(paper + '.png')

In [26]:
def hey(update, context):
    image = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRowHhdTB04EQSOcRuRTqLHvk96K4lS3UupYg&usqp=CAU"
    context.bot.sendPhoto(chat_id=update.effective_chat.id, photo = image, caption = 'Eai Kamako')

SyntaxError: positional argument follows keyword argument (<ipython-input-26-3ce821243718>, line 3)

In [13]:
import pandas as pd # Para evitar escrever pandas e trocar pela escrita apenas de pd para facilitar
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import yfinance as yf 

from pandas_datareader import data as web # Evita a escrita do data e troca pelo web
from datetime import datetime

In [14]:
def get_stock_chart(paper):
    paper_data = yf.download(
    tickers=paper + '.SA', 
    period="1d",
    interval="1m")
    
    plt.plot(paper_data['Close'])
    plt.savefig(paper + '.png')
    plt.clf()
    return paper_data

In [15]:
get_stock_chart('petr4')

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-06-16 10:08:00-03:00,29.080000,29.110001,29.049999,29.110001,29.110001,0
2021-06-16 10:09:00-03:00,29.110001,29.180000,29.090000,29.160000,29.160000,261300
2021-06-16 10:10:00-03:00,29.160000,29.160000,29.070000,29.080000,29.080000,212900
2021-06-16 10:11:00-03:00,29.080000,29.160000,29.070000,29.150000,29.150000,94600
2021-06-16 10:12:00-03:00,29.139999,29.139999,29.070000,29.080000,29.080000,140100
...,...,...,...,...,...,...
2021-06-16 16:51:00-03:00,29.209999,29.290001,29.180000,29.180000,29.180000,1188500
2021-06-16 16:52:00-03:00,29.190001,29.190001,29.120001,29.160000,29.160000,398700
2021-06-16 16:53:00-03:00,29.150000,29.170000,29.139999,29.170000,29.170000,179800


<Figure size 432x288 with 0 Axes>

In [27]:
dispatcher.add_handler(CommandHandler('hello', hello))
dispatcher.add_handler(CommandHandler('dolar', dolar))
dispatcher.add_handler(CommandHandler('euro', euro))
dispatcher.add_handler(CommandHandler('vilzyn', vilzyn))

dispatcher.add_handler(CommandHandler("stock", stock))
dispatcher.add_handler(CommandHandler("chart", chart))
dispatcher.add_handler(CommandHandler("hey", hey))

NameError: name 'hey' is not defined

In [17]:
updater.start_polling()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- VTNC34.SA: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
